# Instalações das bibliotecas

In [1]:
!pip install pytelegrambotapi
!pip install pyUFbr
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyUFbr: filename=pyUFbr-0.1.0-py3-none-any.whl size=63703 sha256=ccb11e6464e2dea59a0eff4a04b73b289960adb527214a4321da80baf3f41868
  Stored in directory: /root/.cache/pip/wheels/b0/5e/a6/df5e053e9847cb01a3031696952ea32a5350c95635bb1d89e3
Successfully built pyUFbr
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.0 MB/s eta 0:00:00


# Chatbot

In [2]:
import telebot
import unidecode
from pyUFbr.baseuf import ufbr
import re

API_TOKEN = '7465156973:AAEUyiaHXYrwqtpoDLjA9_3EPTN2kFR6k70'  # Substitua pelo seu token do Telegram
bot = telebot.TeleBot(API_TOKEN)

user_data = {}

def gerar_link_imovel(tipo, estado, cidade, valor_maximo, tipo_imovel, quartos, banheiro, area_maxima, mobiliado, aceita_pet):
    valor_formatado = valor_maximo.replace("R$", "").replace(".", "").strip()

    link = (f"https://www.quintoandar.com.br/{tipo}/imovel/{cidade.lower().replace(' ', '-')}-{estado.lower()}"
            f"/de-500-a-{valor_formatado}-reais/{tipo_imovel}/{quartos}-quartos/{banheiro}-banheiros/de-50-a-{area_maxima}-m2/{mobiliado}/{aceita_pet}")

    return link

@bot.message_handler(commands=['imovel'])
def iniciar_busca_imovel(message):
    chat_id = message.chat.id
    user_data[chat_id] = {}
    texto = 'Olá, seja bem vindo a imobiliária de Ana e Erika! Você gostaria de alugar ou comprar um imóvel?'
    bot.send_message(chat_id, texto)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'tipo' not in user_data[message.chat.id])
def confirmar_busca_imovel(message):
    chat_id = message.chat.id
    user_data[chat_id]['tipo'] = message.text.lower()
    if "comprar" not in user_data[chat_id]['tipo'] and "alugar" not in user_data[chat_id]['tipo']:
      texto = "Desculpe, não consegui entender. Você gostaria de alugar ou comprar um imóvel?"
      bot.send_message(chat_id, texto)
    else:
      if "comprar" in user_data[chat_id]['tipo']:
        user_data[chat_id]['tipo'] = 'comprar'
      elif "alugar" in user_data[chat_id]['tipo']:
        user_data[chat_id]['tipo'] = 'alugar'
      perguntar_estado_cidade(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'comprar' not in user_data[message.chat.id]['tipo'] and 'alugar' not in user_data[message.chat.id]['tipo'])
def repetir_confirmacao_busca_imovel(message):
  confirmar_busca_imovel(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'estado_cidade' not in user_data[message.chat.id])
def perguntar_estado_cidade(message):
    chat_id = message.chat.id
    user_data[message.chat.id]['estado_cidade'] = 'aguardando'
    texto = 'Em qual estado e cidade? Por favor, informe a sigla do estado(UF) e a cidade'
    bot.send_message(chat_id, texto)

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('estado_cidade') == 'aguardando')
def validar_estado_cidade(message):
    chat_id = message.chat.id
    texto_normalizado = unidecode.unidecode(message.text.lower())

    estado_encontrado = None
    cidade_encontrada = None

    for estado in ufbr.list_uf:
        cidades = ufbr.list_cidades(estado)

        if estado.lower() in texto_normalizado:
            estado_encontrado = estado  # Armazena o estado encontrado

            for cidade in cidades:
                cidade_normalizada = unidecode.unidecode(cidade.lower())
                if cidade_normalizada in texto_normalizado:
                    cidade_encontrada = cidade  # Armazena a cidade encontrada
                    break  # Sai do loop se a cidade for encontrada

    # Verifica se pelo menos um estado e uma cidade foram encontrados
    if not estado_encontrado or not cidade_encontrada:
        texto = "Desculpe, não consegui entender. Por favor, informe a sigla do estado (UF) e a cidade."
        bot.send_message(chat_id, texto)
    else:
        # Se estado e cidade válidos foram encontrados, prossegue para a próxima função
        user_data[chat_id]['estado_cidade'] = (estado_encontrado, cidade_encontrada)  # Armazena os valores
        perguntar_valor_maximo(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'valor_maximo' not in user_data[message.chat.id])
def perguntar_valor_maximo(message):
    chat_id = message.chat.id
    user_data[chat_id]['valor_maximo'] = '0'
    bot.send_message(chat_id, "Qual o valor máximo?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('valor_maximo') == '0')
def confirmar_valor_maximo(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)  # Encontra todos os números

    if len(numeros) != 1:  # Verifica se há exatamente um número
        texto = "Por favor, informe apenas um número máximo."
        bot.send_message(chat_id, texto)
    else:
        user_data[chat_id]['valor_maximo'] = numeros[0]  # Armazena o número como string
        perguntar_tipo_imovel(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'tipo_imovel' not in user_data[message.chat.id])
def perguntar_tipo_imovel(message):
    chat_id = message.chat.id
    user_data[chat_id]['tipo_imovel'] = "aguardando"
    bot.send_message(chat_id, "É apartamento ou casa?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('tipo_imovel') == 'aguardando')
def confirmar_tipo_imovel(message):
    chat_id = message.chat.id
    user_data[chat_id]['tipo_imovel'] = message.text.lower()
    if "apartamento" not in user_data[chat_id]['tipo_imovel'] and "casa" not in user_data[chat_id]['tipo_imovel']:
      texto = "Desculpe, não consegui entender. Você gostaria de uma casa ou um apartamento?"
      bot.send_message(chat_id, texto)
    else:
      if "apartamento" in user_data[chat_id]['tipo_imovel']:
        user_data[chat_id]['tipo_imovel'] = 'apartamento'
      elif "casa" in user_data[chat_id]['tipo_imovel']:
        user_data[chat_id]['tipo_imovel'] = 'casa'
      perguntar_quartos(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'apartamento' not in user_data[message.chat.id]['tipo_imovel'] and 'casa' not in user_data[message.chat.id]['tipo_imovel'])
def repetir_confirmacao_tipo_imovel(message):
  confirmar_tipo_imovel(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'quartos' not in user_data[message.chat.id])
def perguntar_quartos(message):
    chat_id = message.chat.id
    user_data[chat_id]['quartos'] = "0"
    bot.send_message(chat_id, "Quantos quartos?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('quartos') == '0')
def confirmar_quartos(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)  # Encontra todos os números

    if len(numeros) != 1:  # Verifica se há exatamente um número
        texto = "Por favor, informe apenas um número."
        bot.send_message(chat_id, texto)
    else:
        user_data[chat_id]['quartos'] = numeros[0]  # Armazena o número como string
        perguntar_banheiro(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'banheiro' not in user_data[message.chat.id])
def perguntar_banheiro(message):
    chat_id = message.chat.id
    user_data[chat_id]['banheiro'] = "0"
    bot.send_message(chat_id, "Quantos banheiros?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('banheiro') == '0')
def confirmar_banheiro(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)  # Encontra todos os números

    if len(numeros) != 1:  # Verifica se há exatamente um número
        texto = "Por favor, informe apenas um número."
        bot.send_message(chat_id, texto)

    elif int(numeros[0]) < 1 or int(numeros[0]) > 4:
        texto = "Por favor, informe um número entre 1 e 4."
        bot.send_message(chat_id, texto)
        return

    else:
      banheiros = []
      for i in range(int(numeros[0]), 5):  # Cria uma lista de números de 'numero' até 4
          banheiros.append(str(i))  # Adiciona cada número como string

      user_data[chat_id]['banheiro'] = '-'.join(banheiros)  # Armazena a string no formato desejado
      perguntar_area_maxima(message)  # Chama a função após armazenar

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'area_maxima' not in user_data[message.chat.id])
def perguntar_area_maxima(message):
    chat_id = message.chat.id
    user_data[chat_id]['area_maxima'] = "0"
    bot.send_message(chat_id, "Qual a área máxima?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('area_maxima') == '0')
def confirmar_area_maxima(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)  # Encontra todos os números

    if len(numeros) != 1:  # Verifica se há exatamente um número
        texto = "Por favor, informe apenas um número máximo."
        bot.send_message(chat_id, texto)
    else:
        user_data[chat_id]['area_maxima'] = numeros[0]  # Armazena o número como string
        perguntar_mobilia(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'mobiliado' not in user_data[message.chat.id])
def perguntar_mobilia(message):
    chat_id = message.chat.id
    user_data[chat_id]['mobiliado'] = "0"
    bot.send_message(chat_id, "Você quer que esteja mobiliado? Sim/Não")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('mobiliado') == '0')
def confirmar_mobilia(message):
    chat_id = message.chat.id
    user_data[chat_id]['mobiliado'] = message.text.lower()
    if "sim" not in user_data[chat_id]['mobiliado'] and "nao" not in user_data[chat_id]['mobiliado']:
      texto = "Desculpe, não consegui entender. Você gostaria de mobília? Sim/Não"
      bot.send_message(chat_id, texto)
    else:
      if "sim" in user_data[chat_id]['mobiliado']:
        user_data[chat_id]['mobiliado'] = 'nao-mobiliado'
      elif "nao" in user_data[chat_id]['mobiliado']:
        user_data[chat_id]['mobiliado'] = 'mobiliado'
      perguntar_aceita_pet(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'mobiliado' not in user_data[message.chat.id]['mobiliado'] and 'nao-mobiliado' not in user_data[message.chat.id]['mobiliado'])
def repetir_confirmacao_mobilia(message):
  confirmar_mobilia(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'aceita_pet' not in user_data[message.chat.id])
def perguntar_aceita_pet(message):
    chat_id = message.chat.id
    user_data[chat_id]['aceita_pet'] = "0"
    bot.send_message(chat_id, "Você quer que aceite pet? Sim/Não")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('aceita_pet') == '0')
def confirmar_aceita_pet(message):
    chat_id = message.chat.id
    user_data[chat_id]['aceita_pet'] = message.text.lower()

    if "sim" not in user_data[chat_id]['aceita_pet'] and "nao" not in user_data[chat_id]['aceita_pet']:
      texto = "Desculpe, não consegui entender. Você gostaria que aceitasse pet? Sim/Não"
      bot.send_message(chat_id, texto)
    else:
      if "sim" in user_data[chat_id]['aceita_pet']:
        user_data[chat_id]['aceita_pet'] = 'aceita-pets'
      elif "nao" in user_data[chat_id]['aceita_pet']:
        user_data[chat_id]['mobiliado'] = 'nao-aceita-pets'
      gerar_link_final(message)


@bot.message_handler(func=lambda message: message.chat.id in user_data and 'aceita-pet' not in user_data[message.chat.id]['aceita_pet'] and 'nao-aceita-pet' not in user_data[message.chat.id]['aceita_pet'])
def repetir_confirmacao_aceita_pet(message):
  confirmar_aceita_pet(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'banheiro' in user_data[message.chat.id])
def gerar_link_final(message):
    chat_id = message.chat.id

    tipo = user_data[chat_id]['tipo']
    estado, cidade = user_data[chat_id]['estado_cidade']
    valor_maximo = user_data[chat_id]['valor_maximo']
    tipo_imovel = user_data[chat_id]['tipo_imovel']
    quartos = user_data[chat_id]['quartos']
    banheiro = user_data[chat_id]['banheiro']
    area_maxima = user_data[chat_id]['area_maxima']
    mobiliado = user_data[chat_id]['mobiliado']
    aceita_pet = user_data[chat_id]['aceita_pet']


    link_imovel = gerar_link_imovel(tipo, estado, cidade, valor_maximo, tipo_imovel, quartos, banheiro, area_maxima, mobiliado, aceita_pet)

    bot.send_message(chat_id, f"Aqui está o link para imóveis em {cidade}:\n{link_imovel}")

    del user_data[chat_id]

Ativação do Chatbot

In [ ]:
bot.polling()